In [1]:
from importlib import reload

import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.cm as cm

import re

import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

C:\Users\339340273\AppData\Local\Continuum\anaconda3\envs\gaku\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
import probscale
import seaborn as sns

In [3]:
import sys
sys.path.append('C:/dstools')
import sig_test

In [4]:
from pyhive import presto
req_kw = {
  'verify': 'C:/presto/Presto_JDBC_Driver/WDC_CA_bundle.pem'
}

# ETL Script

In [5]:
def get_ghl2_table (table, searchCond, sel, whereAdded = None, fName=None):
    
    con = presto.connect(
      host='bdp-e2e-presto.wdc.com',
      port=8446,
      protocol='https',
      catalog='hive',
      username='gaku.kiuchi@wdc.com',
      requests_kwargs=req_kw
    )

    batchsize = 100

    df = pd.DataFrame({})
    df_sn = pd.read_csv(searchCond[0])
    for strt in range(0, df_sn['sn'].shape[0], batchsize):
        sql = "select %s " % sel
        sql += "from ghl2."+table + " "
        sql += "where product='pdq' "
        sql += "and testcode in (%s) "  % re.sub('[\[\]]', '', '%s' % searchCond[1])
        sql += "and enddt between '%s' and '%s' " % (searchCond[2][0], searchCond[2][1]) 
        #sql += "and hddtrial in (%s) "  % re.sub('[\[\]]', '', '%s' % searchCond[3])    
        sql += "and hddsn in (%s) "     % re.sub('[\[\]]', '', '%s' % list(df_sn['sn'].values)[strt:strt+batchsize])
        sql += "and procid in (%s) "    % re.sub('[\[\]]', '', '%s' % searchCond[4])
        sql += "and qualifier in (%s) " % re.sub('[\[\]]', '', '%s' % searchCond[5])
        sql += "and pfcode not in ('9999', '99V6', '99V7') "
        if whereAdded != None:
            sql += "and %s " % whereAdded
        print ('current sql is %s' % sql)
        df = df.append( pd.read_sql(sql, con) )
        print(df.shape)

    con.close()
    
    if fName == None:
        df.to_pickle('df_%s.pkl'%table)
        df.to_csv('df_%s.csv'%table, index=False)
    else:
        df.to_pickle('%s.pkl'%fName)
        df.to_csv('%s.csv'%fName, index=False)
    return(df)

In [6]:
def get_vqaa_table (table, searchCond, sel, whereAdded = None):
    
    con = presto.connect(
      host='bdp-e2e-presto.wdc.com',
      port=8446,
      protocol='https',
      catalog='hive',
      username='gaku.kiuchi@wdc.com',
      requests_kwargs=req_kw
    )

    batchsize = 100

    df = pd.DataFrame({})
    df_sn = pd.read_csv(searchCond[0])
    for strt in range(0, df_sn['sn'].shape[0], batchsize):
        sql = "select %s " % sel
        sql += "from vqaa."+table + " "
        sql += "where product='pdq' "
        sql += "and testpgmver in (%s) "  % re.sub('[\[\]]', '', '%s' % searchCond[1])
        sql += "and enddt between '%s' and '%s' " % (searchCond[2][0], searchCond[2][1]) 
        sql += "and hddtrial in (%s) "  % re.sub('[\[\]]', '', '%s' % searchCond[3])    
        sql += "and hddsn in (%s) "     % re.sub('[\[\]]', '', '%s' % list(df_sn['sn'].values)[strt:strt+batchsize])
        sql += "and procid in (%s) "    % re.sub('[\[\]]', '', '%s' % searchCond[4])
        #sql += "and qualifier in (%s) " % re.sub('[\[\]]', '', '%s' % searchCond[5])
        #sql += "and pfcode not in ('9999', '99V6', '99V7') "
        if whereAdded != None:
            sql += "and %s " % whereAdded
        print ('current sql is %s' % sql)
        df = df.append( pd.read_sql(sql, con) )
        print(df.shape)

    con.close()
        
    df.to_pickle('df_%s.pkl'%table)
    df.to_csv('df_%s.csv'%table, index=False)
    return(df)

In [7]:
def get_sdet_table (table, searchCond, sel, whereAdded = None):
    
    con = presto.connect(
      host='bdp-e2e-presto.wdc.com',
      port=8446,
      protocol='https',
      catalog='hive',
      username='gaku.kiuchi@wdc.com',
      requests_kwargs=req_kw
    )

    batchsize = 100

    df = pd.DataFrame({})
    df_sn = pd.read_csv(searchCond[0])
    for strt in range(0, df_sn['slidersn'].shape[0], batchsize):
        sql = "select %s " % sel
        sql += "from hive."+table + " "
        sql += "where "
        sql += "storeday between '%s' and '%s' " % (searchCond[1][0], searchCond[1][1])
        sql += "and slidersn in (%s) " % re.sub('[\[\]]', '', '%s' % list(df_sn['slidersn'].values)[strt:strt+batchsize])
        if whereAdded != None:
            sql += "and %s " % whereAdded
        print ('current sql is %s' % sql)
        df = df.append( pd.read_sql(sql, con) )
        print(df.shape)

    con.close()
        
    df.to_pickle('df_%s.pkl'%table)
    df.to_csv('df_%s.csv'%table, index=False)
    return(df)

# ETL

In [8]:
#parameters
stpi      = 580.0
dtpi_cmr  = 500.0
dtpi_smr  = 602.0
tgtLbaCmr = 273906721.0
tgtLbaSmr = 333662709.0

In [9]:
#peak current

def IwPeak4G4(iw, ka, kd):
    return((-0.013954886*ka + 1.128161219) * iw + (1.973010793 * ka -1.407847328))

def IwPeak4G4i(iw, ka, kd):
    return((-0.015470889*ka + 1.684161818) * iw + (1.335027916 * ka +0.387637386))


In [10]:
def get_nth_char (strList, index=0):
    a = []
    for q in list(strList):
        a.append(q[index])
    return(a)

In [11]:
snlist_csv   = 'sn_all.csv'
listTestcode = ['PDQX083M']
listEnddt    = ['20201228', '20210107']
listHddTrial = ['Z07E']

# Head Map Check

In [12]:
#Head Map
inputParamGhl2Rmr = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6400'],
                     ['10N0']
                    ]
dfHeadMap = get_ghl2_table('ccb_ci_rmr', inputParamGhl2Rmr, 
                           'hddsn, testcode, mfgid, mfgid_5, hddtrial, lhd, phd, \
                            row_number() over ( partition by hddsn, testcode, lhd order by enddate desc) as row_num')
dfHeadMap.to_pickle('dfHeadMap.pkl')
dfHeadMap = pd.read_pickle('dfHeadMap.pkl').query("row_num==1")
dfHeadMap

current sql is select hddsn, testcode, mfgid, mfgid_5, hddtrial, lhd, phd,                             row_number() over ( partition by hddsn, testcode, lhd order by enddate desc) as row_num from ghl2.ccb_ci_rmr where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('10N0') and pfcode not in ('9999', '99V6', '99V7') 
(64, 8)


,hddsn,testcode,mfgid,mfgid_5,hddtrial,lhd,phd,row_num
0,2FA00EPA,PDQX083M,KJ1E03,3,Z07E,4,10,1
1,2FA01K0A,PDQX083M,KJBE03,3,Z07E,4,10,1
2,2FA026TA,PDQX083M,KJ1E03,3,Z07E,5,11,1
3,2FA00EWA,PDQX083M,KJ1E03,3,Z07E,1,1,1
4,2FA026TA,PDQX083M,KJ1E03,3,Z07E,7,17,1
...,...,...,...,...,...,...,...,...
59,2FA00ESA,PDQX083M,KJ1E03,3,Z07E,7,17,1
60,2FA00EPA,PDQX083M,KJ1E03,3,Z07E,1,1,1
61,2FA00EYA,PDQX083M,KJ1E03,3,Z07E,1,1,1
62,2FA01K0A,PDQX083M,KJBE03,3,Z07E,1,1,1


In [13]:
pd.pivot_table(dfHeadMap.query("lhd==0"), index=['hddsn', 'mfgid_5'], columns=['testcode'], values='hddtrial', aggfunc='count')

,testcode,PDQX083M
hddsn,mfgid_5,
2FA00E8A,3,1
2FA00EPA,3,1
2FA00ESA,3,1
2FA00EWA,3,1
2FA00EYA,3,1
2FA01HYA,3,1
2FA01K0A,3,1
2FA026TA,3,1


# SER, OW, Pbo

In [14]:
## SER ##
inputParamGhl2Ser = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6400', '6600', '6800'],
                     ['1000', '1010', 
                      '2000', '2010',
                      '30N0', '3050', '3060', 
                      '40N0', '4050', '4060', 
                      '9070', '9050', '9060', 
                      'K070', 'K050', 'K060',
                      'B0N0', 'B010', 'B020', 
                      'M0N0', 'M010', 'M020',
                     ]
                     #['1000', '1010', '1020', 
                     # '10A0',
                     # '30N0', '3050', '3060']
                    ]
dfSer = get_ghl2_table('ccb_ci_ser', inputParamGhl2Ser, 
                       'hddsn, testcode, qualifier, mfgid, hddtrial, lhd, phd, band, ser, offset, \
                        row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num ',
                       'band < 64'
                      )
dfSer

current sql is select hddsn, testcode, qualifier, mfgid, hddtrial, lhd, phd, band, ser, offset,                         row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num  from ghl2.ccb_ci_ser where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400', '6600', '6800') and qualifier in ('1000', '1010', '2000', '2010', '30N0', '3050', '3060', '40N0', '4050', '4060', '9070', '9050', '9060', 'K070', 'K050', 'K060', 'B0N0', 'B010', 'B020', 'M0N0', 'M010', 'M020') and pfcode not in ('9999', '99V6', '99V7') and band < 64 
(83968, 11)


,hddsn,testcode,qualifier,mfgid,hddtrial,lhd,phd,band,ser,offset,row_num
0,2FA00E8A,PDQX083M,1010,KJ1E03,Z07E,3,9,29,-2.98,-35,1
1,2FA00E8A,PDQX083M,2000,KJ1E03,Z07E,0,0,44,-2.99,434,1
2,2FA00E8A,PDQX083M,4050,KJ1E03,Z07E,0,0,41,-2.22,71,1
3,2FA00E8A,PDQX083M,9050,KJ1E03,Z07E,6,16,13,-1.78,-69,1
4,2FA00E8A,PDQX083M,9070,KJ1E03,Z07E,7,17,8,-2.20,-158,1
...,...,...,...,...,...,...,...,...,...,...,...
83963,2FA026TA,PDQX083M,K070,KJ1E03,Z07E,7,17,14,-2.08,-64,1
83964,2FA026TA,PDQX083M,B010,KJ1E03,Z07E,3,9,29,-1.96,16,1
83965,2FA026TA,PDQX083M,B0N0,KJ1E03,Z07E,5,11,6,-2.17,-235,1
83966,2FA026TA,PDQX083M,M010,KJ1E03,Z07E,0,0,31,-2.10,50,1


In [15]:
## Roller SER ##
inputParamGhl2SerRoller = [snlist_csv, 
                           listTestcode,
                           listEnddt, 
                           listHddTrial,
                           ['6600', '6800'],
                           ['9040', 'B000', 'K040', 'M000']
                          ]
dfSerRoller = get_ghl2_table('ccb_ci_ser', inputParamGhl2SerRoller, 
                       'hddsn, testcode, qualifier, mfgid, hddtrial, lhd, phd, band, offset, \
                        aveperband, maxperband, minperband, rawmodulationstats0_average as aveperband0, rawmodulationstats0_max as maxperband0, rawmodulationstats0_min as minperband0, \
                        row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num ',
                       'band < 64',
                       fName='df_ser_roller'
                      )
dfSerRoller

current sql is select hddsn, testcode, qualifier, mfgid, hddtrial, lhd, phd, band, offset,                         aveperband, maxperband, minperband, rawmodulationstats0_average as aveperband0, rawmodulationstats0_max as maxperband0, rawmodulationstats0_min as minperband0,                         row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num  from ghl2.ccb_ci_ser where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6600', '6800') and qualifier in ('9040', 'B000', 'K040', 'M000') and pfcode not in ('9999', '99V6', '99V7') and band < 64 
(224, 16)


,hddsn,testcode,qualifier,mfgid,hddtrial,lhd,phd,band,offset,aveperband,maxperband,minperband,aveperband0,maxperband0,minperband0,row_num
0,2FA00ESA,PDQX083M,9040,KJ1E03,Z07E,2,8,0,-128,-0.986966,-0.907448,-1.190141,-0.98,-0.90,-1.19,1
1,2FA00E8A,PDQX083M,B000,KJ1E03,Z07E,5,11,0,-124,-0.977736,-0.902384,-1.130681,-0.97,-0.90,-1.13,1
2,2FA01HYA,PDQX083M,M000,KJBE03,Z07E,5,11,0,160,-0.758434,-0.670008,-0.970214,-0.75,-0.67,-0.97,1
3,2FA00EYA,PDQX083M,B000,KJ1E03,Z07E,5,11,0,-60,-1.060276,-0.970171,-1.267394,-1.06,-0.97,-1.26,1
4,2FA01K0A,PDQX083M,9040,KJBE03,Z07E,3,9,0,-110,-1.003289,-0.936341,-1.180156,-1.00,-0.93,-1.18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2FA01HYA,PDQX083M,K040,KJBE03,Z07E,3,9,0,101,-0.895626,-0.839925,-1.010157,-0.89,-0.83,-1.01,1
220,2FA00ESA,PDQX083M,B000,KJ1E03,Z07E,2,8,0,-106,-1.039504,-0.948272,-1.214252,-1.03,-0.94,-1.21,1
221,2FA01HYA,PDQX083M,B000,KJBE03,Z07E,0,0,0,-127,-1.059609,-0.976920,-1.243889,-1.05,-0.97,-1.24,1
222,2FA01HYA,PDQX083M,M000,KJBE03,Z07E,6,16,0,-226,-0.964836,-0.868400,-1.094536,-0.96,-0.86,-1.09,1


In [16]:
## OW ##
inputParamGhl2Ow = [snlist_csv, 
                    listTestcode,
                    listEnddt, 
                    listHddTrial,
                    ['6400', '6600', '6800'],
                    ['11N0', '21N0', '31N0', '41N0', '91H0', 'K1H0', 'B0N0', 'M0N0']
                   ]
dfOw = get_ghl2_table('ccb_ci_ow', inputParamGhl2Ow, 
                      'hddsn, enddt, pfcode, mfgid, hddtrial, testcode, qualifier, lhd, phd, band, owvalperp, owvalmaxperp, owvalconv, owvalmaxconv, \
                       row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                      'band < 3'
                     )
dfOw = pd.read_pickle('df_ccb_ci_ow.pkl').query("band < 3 and row_num==1")


current sql is select hddsn, enddt, pfcode, mfgid, hddtrial, testcode, qualifier, lhd, phd, band, owvalperp, owvalmaxperp, owvalconv, owvalmaxconv,                        row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_ow where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400', '6600', '6800') and qualifier in ('11N0', '21N0', '31N0', '41N0', '91H0', 'K1H0', 'B0N0', 'M0N0') and pfcode not in ('9999', '99V6', '99V7') and band < 3 
(1440, 15)


In [17]:
## PBO ##
inputParamGhl2Ow = [snlist_csv, 
                    listTestcode,
                    listEnddt, 
                    listHddTrial,
                    ['6400'],
                    ['3070', '4070']
                   ]
dfPbo = get_ghl2_table('ccb_mi_pbo', inputParamGhl2Ow, 
                      'hddsn, enddt, pfcode, mfgid, hddtrial, testcode, qualifier, lhd, phd, band, mRwSerPerRWNmSlope, mRwSerPerRWDacOffset, \
                       row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                      'band = 0 '
                     )
dfPbo

current sql is select hddsn, enddt, pfcode, mfgid, hddtrial, testcode, qualifier, lhd, phd, band, mRwSerPerRWNmSlope, mRwSerPerRWDacOffset,                        row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_mi_pbo where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('3070', '4070') and pfcode not in ('9999', '99V6', '99V7') and band = 0  
(128, 13)


,hddsn,enddt,pfcode,mfgid,hddtrial,testcode,qualifier,lhd,phd,band,mRwSerPerRWNmSlope,mRwSerPerRWDacOffset,row_num
0,2FA00EWA,20201228,0000,KJ1E03,Z07E,PDQX083M,3070,5,11,0,319.02713,-223.73526,1
1,2FA00ESA,20201228,0000,KJ1E03,Z07E,PDQX083M,3070,0,0,0,424.31894,-244.19183,1
2,2FA01HYA,20201228,0000,KJBE03,Z07E,PDQX083M,3070,3,9,0,451.03296,-226.70676,1
3,2FA01HYA,20201228,0000,KJBE03,Z07E,PDQX083M,4070,6,16,0,211.85587,-212.88670,1
4,2FA00E8A,20201228,0000,KJ1E03,Z07E,PDQX083M,4070,2,8,0,131.82375,-205.34700,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2FA00E8A,20201228,0000,KJ1E03,Z07E,PDQX083M,3070,7,17,0,511.00540,-218.12600,1
124,2FA00EPA,20201228,0000,KJ1E03,Z07E,PDQX083M,4070,5,11,0,416.37936,-210.59515,1
125,2FA01HYA,20201228,0000,KJBE03,Z07E,PDQX083M,4070,7,17,0,187.88443,-217.66391,1
126,2FA00EYA,20201228,0000,KJ1E03,Z07E,PDQX083M,3070,2,8,0,265.78717,-218.00385,1


In [18]:
#TSW2 spec
# 0.0457 * OWC_Worst + 904_ser_roller_max + (0.16+0.194)*u407_rwser_rw_slp_nm_od*1e-3 > -1.5537
#TSWH spec
# 0.0065 * OWC_Worst + k04_ser_roller_max + (0.16+0.194)*u407_rwser_rw_slp_nm_od*1e-3 > -0.66903
coef = [0.0457, 0.0065];      #CMR, SMR
thresh = [-1.5537, -1.6286, -0.66903, -0.68404]; #CMR for SRST, CMR for Final, SMR for SRST, SMR for Final
dfh4S  = 0.16+0.194;     #nm, for SRST
dfh4F  = 0.324;          #nm, for Final

#OW
dfOw = pd.read_pickle('df_ccb_ci_ow.pkl').query("row_num==1 and band==0")
dfOw.loc[:,'firstQual'] = get_nth_char(list(dfOw.qualifier), index=0)
#SER
dfSer = pd.read_pickle('df_ser_roller.pkl').query("row_num==1 and band==0")
dfSer.loc[:,'firstQual'] = get_nth_char(list(dfSer.qualifier), index=0)
#Pbo
dfPbo = pd.read_pickle('df_ccb_mi_pbo.pkl').query("row_num==1 and band==0")

dfTsw = dfOw.query("band==0 and qualifier in ['91H0', 'K1H0', 'B0N0', 'M0N0']").merge(dfSer.query("qualifier in ['9040', 'K040', 'B000', 'M000']"), 
                                                                   left_on =['hddsn', 'mfgid','hddtrial', 'testcode', 'firstQual', 'lhd', 'phd', 'band', 'row_num'],
                                                                   right_on=['hddsn', 'mfgid','hddtrial', 'testcode', 'firstQual', 'lhd', 'phd', 'band', 'row_num'],
                                                                   suffixes=['_ow', '_ser'],
                                                                   how='inner'
                                                                  )
dfTsw=dfTsw.merge(dfPbo,
                  left_on =['hddsn', 'mfgid','hddtrial', 'testcode', 'lhd', 'phd', 'band', 'row_num'],
                  right_on=['hddsn', 'mfgid','hddtrial', 'testcode', 'lhd', 'phd', 'band', 'row_num'],
                  suffixes=['', '_pbo'],
                  how='left'
                 )

#ser with dfh
#SRST
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('9') | dfTsw.qualifier_ser.str.startswith('K'), 'dfh'] = dfh4S
#Final
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('B') | dfTsw.qualifier_ser.str.startswith('M'), 'dfh'] = dfh4F
dfTsw.loc[:,'ser_for_tsw_metric'] = dfTsw.loc[:,'maxperband'].values + dfTsw.loc[:,'mRwSerPerRWNmSlope'].values*dfTsw.loc[:,'dfh'].values*1e-3

#coef
#CMR
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('9') | dfTsw.qualifier_ser.str.startswith('B'),'coef'] = coef[0]
#SMR
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('K') | dfTsw.qualifier_ser.str.startswith('M'),'coef'] = coef[1]

#spec threshold
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('9'), 'thresh'] = thresh[0]
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('B'), 'thresh'] = thresh[1]
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('K'), 'thresh'] = thresh[2]
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('M'), 'thresh'] = thresh[3]

#TSW Metric
dfTsw.loc[:,'tsw_metric'] = dfTsw.loc[:,'owvalmaxconv'].values * dfTsw.loc[:,'coef'].values + dfTsw.loc[:,'ser_for_tsw_metric'].values

dfTsw.to_pickle('dfTsw.pkl')
dfTsw.to_csv('dfTsw.csv')

dfTsw

,hddsn,enddt,pfcode,mfgid,hddtrial,testcode,qualifier_ow,lhd,phd,band,...,enddt_pbo,pfcode_pbo,qualifier,mRwSerPerRWNmSlope,mRwSerPerRWDacOffset,dfh,ser_for_tsw_metric,coef,thresh,tsw_metric
0,2FA01K0A,20210105,0703,KJBE03,Z07E,PDQX083M,91H0,3,9,0,...,20201228,0000,3070,289.95670,-227.35725,0.354,-0.833696,0.0457,-1.55370,-2.351393
1,2FA01K0A,20210105,0703,KJBE03,Z07E,PDQX083M,91H0,3,9,0,...,20201228,0000,4070,212.99448,-222.55740,0.354,-0.860941,0.0457,-1.55370,-2.378638
2,2FA026TA,20210105,0000,KJ1E03,Z07E,PDQX083M,K1H0,2,8,0,...,20201228,0000,3070,488.38535,-214.71205,0.354,-0.616811,0.0065,-0.66903,-0.804271
3,2FA026TA,20210105,0000,KJ1E03,Z07E,PDQX083M,K1H0,2,8,0,...,20201228,0000,4070,406.34973,-222.71560,0.354,-0.645851,0.0065,-0.66903,-0.833311
4,2FA00ESA,20210106,6DRM,KJ1E03,Z07E,PDQX083M,B0N0,7,17,0,...,20201228,0000,4070,169.55612,-215.34903,0.324,-0.850890,0.0457,-1.62860,-2.256165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,2FA01HYA,20210106,6DRM,KJBE03,Z07E,PDQX083M,M0N0,5,11,0,...,20201228,0000,4070,306.34778,-203.83963,0.324,-0.570752,0.0065,-0.68404,-0.745732
444,2FA026TA,20210105,0000,KJ1E03,Z07E,PDQX083M,91H0,3,9,0,...,20201228,0000,4070,188.30003,-215.31906,0.354,-0.719245,0.0457,-1.55370,-2.168849
445,2FA026TA,20210105,0000,KJ1E03,Z07E,PDQX083M,91H0,3,9,0,...,20201228,0000,3070,340.92470,-214.45732,0.354,-0.665216,0.0457,-1.55370,-2.114820
446,2FA00E8A,20210105,0000,KJ1E03,Z07E,PDQX083M,91H0,2,8,0,...,20201228,0000,4070,131.82375,-205.34700,0.354,-0.797092,0.0457,-1.55370,-2.314332


# RSBN

In [19]:
## RSBN ##

inputParamGhl2Rsbn = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400'],
                      ['1000', '2010']
                     ]
dfRsbn = get_ghl2_table('ccb_ci_crsbn_datareader', inputParamGhl2Rsbn, 
                        'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, phd, lhd, readerindex, datardrs, pridatardr, \
                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, readerindex order by enddate desc) as row_num',
                        'readerindex < 64'
                       )
dfRsbn = pd.read_pickle('df_ccb_ci_crsbn_datareader.pkl')
dfRsbn.loc[:,'datardrs']   = dfRsbn.loc[:,'datardrs'].values.astype(np.int64)
dfRsbn.loc[:,'pridatardr'] = dfRsbn.loc[:,'pridatardr'].values.astype(np.int64)
dfRsbnFlat = pd.pivot_table(dfRsbn.query("readerindex<64 and row_num==1"), 
                            index=['hddsn', 'mfgid', 'testcode', 'lhd', 'phd'], 
                            columns=['qualifier', 'readerindex'], 
                            values=['datardrs', 'pridatardr'], 
                            aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfRsbnFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfRsbnFlat.columns)[5:]]
dfRsbnFlat.columns = cols
dfRsbnFlat.to_pickle('dfRsbnFlat.pkl')
dfRsbnFlat

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, phd, lhd, readerindex, datardrs, pridatardr,                          row_number() over ( partition by hddsn, testcode, qualifier, lhd, readerindex order by enddate desc) as row_num from ghl2.ccb_ci_crsbn_datareader where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('1000', '2010') and pfcode not in ('9999', '99V6', '99V7') and readerindex < 64 
(8192, 12)


,hddsn,mfgid,testcode,lhd,phd,datardrs_1000_0,datardrs_1000_1,datardrs_1000_2,datardrs_1000_3,datardrs_1000_4,...,pridatardr_2010_54,pridatardr_2010_55,pridatardr_2010_56,pridatardr_2010_57,pridatardr_2010_58,pridatardr_2010_59,pridatardr_2010_60,pridatardr_2010_61,pridatardr_2010_62,pridatardr_2010_63
0,2FA00E8A,KJ1E03,PDQX083M,0,0,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
1,2FA00E8A,KJ1E03,PDQX083M,1,1,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
2,2FA00E8A,KJ1E03,PDQX083M,2,8,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
3,2FA00E8A,KJ1E03,PDQX083M,3,9,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
4,2FA00E8A,KJ1E03,PDQX083M,4,10,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2FA026TA,KJ1E03,PDQX083M,3,9,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
60,2FA026TA,KJ1E03,PDQX083M,4,10,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
61,2FA026TA,KJ1E03,PDQX083M,5,11,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
62,2FA026TA,KJ1E03,PDQX083M,6,16,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0


In [20]:
## MCW ##
inputParamGhl2Mcw = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400'],
                      ['10N0', '2300']
                     ]
dfMcw = get_ghl2_table('ccb_ci_mcw', inputParamGhl2Mcw, 
                        'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, mcw, mcw_smr, \
                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num')
dfMcw = pd.read_pickle('df_ccb_ci_mcw.pkl').query("row_num==1")
dfMcw.loc[:,'mcw_nm']     = dfMcw.loc[:,'mcw'].values     * (1/100*25.4/dtpi_cmr*1e3)
dfMcw.loc[:,'mcw_smr_nm'] = dfMcw.loc[:,'mcw_smr'].values * (1/4096*25.4/dtpi_smr*1e3)
dfMcw.to_pickle('dfMcw.pkl')
dfMcw

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, mcw, mcw_smr,                          row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_mcw where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('10N0', '2300') and pfcode not in ('9999', '99V6', '99V7') 
(640, 12)


,hddsn,enddt,testcode,mfgid,hddtrial,qualifier,band,phd,lhd,mcw,mcw_smr,row_num,mcw_nm,mcw_smr_nm
0,2FA00EPA,20201228,PDQX083M,KJ1E03,Z07E,10N0,2,0,0,119.531250,0,1,60.721875,0.000000
1,2FA00E8A,20201228,PDQX083M,KJ1E03,Z07E,2300,0,17,7,0.000000,5377,1,0.000000,55.388208
2,2FA00ESA,20201228,PDQX083M,KJ1E03,Z07E,2300,1,8,2,0.000000,6134,1,0.000000,63.186027
3,2FA00ESA,20201228,PDQX083M,KJ1E03,Z07E,2300,3,8,2,0.000000,5952,1,0.000000,61.311254
4,2FA00EYA,20201228,PDQX083M,KJ1E03,Z07E,2300,4,0,0,0.000000,5923,1,0.000000,61.012527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,2FA01HYA,20201228,PDQX083M,KJBE03,Z07E,10N0,1,8,2,108.300781,0,1,55.016797,0.000000
636,2FA00E8A,20201228,PDQX083M,KJ1E03,Z07E,10N0,2,1,1,129.077148,0,1,65.571191,0.000000
637,2FA026TA,20201228,PDQX083M,KJ1E03,Z07E,2300,1,8,2,0.000000,5286,1,0.000000,54.450821
638,2FA01K0A,20201228,PDQX083M,KJBE03,Z07E,2300,0,16,6,0.000000,6234,1,0.000000,64.216122


In [21]:
## MRW/MWW ##
inputParamGhl2Mrw = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400'],
                      ['1000', '1010', '2000', '2010']
                     ]
dfMrw = get_ghl2_table('ccb_ci_mrw', inputParamGhl2Mrw, 
                       'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, mrw, mww, \
                        row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                       'band < 3'
                      )
dfMrw = pd.read_pickle('df_ccb_ci_mrw.pkl').query("row_num==1")
dfMrw.loc[dfMrw.qualifier.str.startswith('1'), 'scale'] = 1/100*25.4/dtpi_cmr*1e3
dfMrw.loc[dfMrw.qualifier.str.startswith('2'), 'scale'] = 1/100*25.4/dtpi_smr*1e3
dfMrw.loc[:,'mrw_nm'] = dfMrw.loc[:,'mrw'].values * dfMrw.loc[:, 'scale'].values
dfMrw.loc[:,'mww_nm'] = dfMrw.loc[:,'mww'].values * dfMrw.loc[:, 'scale'].values
dfMrw.to_pickle('dfMrw.pkl')
dfMrw

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, mrw, mww,                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_mrw where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('1000', '1010', '2000', '2010') and pfcode not in ('9999', '99V6', '99V7') and band < 3 
(768, 12)


,hddsn,enddt,testcode,mfgid,hddtrial,qualifier,band,phd,lhd,mrw,mww,row_num,scale,mrw_nm,mww_nm
0,2FA026TA,20201228,PDQX083M,KJ1E03,Z07E,2000,1,10,4,64.182440,111.496800,1,0.421927,27.080299,47.043500
1,2FA00EWA,20201228,PDQX083M,KJ1E03,Z07E,2010,0,9,3,65.592514,121.123120,1,0.421927,27.675247,51.105104
2,2FA00EWA,20201228,PDQX083M,KJ1E03,Z07E,2000,0,1,1,58.464073,115.495870,1,0.421927,24.667566,48.730816
3,2FA01HYA,20201228,PDQX083M,KJBE03,Z07E,2000,2,16,6,61.021720,119.265810,1,0.421927,25.746706,50.321455
4,2FA01K0A,20201228,PDQX083M,KJBE03,Z07E,1000,1,8,2,52.751984,105.384110,1,0.508000,26.798008,53.535128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,2FA00EWA,20201228,PDQX083M,KJ1E03,Z07E,2010,0,8,2,66.130870,129.697340,1,0.421927,27.902394,54.722798
764,2FA00EYA,20201228,PDQX083M,KJ1E03,Z07E,1010,2,0,0,57.183784,99.700966,1,0.508000,29.049362,50.648091
765,2FA00E8A,20201228,PDQX083M,KJ1E03,Z07E,1000,2,10,4,56.520073,101.832520,1,0.508000,28.712197,51.730920
766,2FA00EPA,20201228,PDQX083M,KJ1E03,Z07E,2000,1,8,2,66.018870,127.299350,1,0.421927,27.855138,53.711021


In [22]:
## AMP ##
inputParamGhl2Amp = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400', '6600', '6800'],
                      ['1000', '1010', '2000', '2010', 
                       '3000', '3010', '4000', '4010', 
                       '9000', '9010', 'K000', 'K010',
                       'B000', 'B010', 'M000', 'M010' 
                      ]
                     ]
dfAmp = get_ghl2_table('ccb_ci_amp', 
                       inputParamGhl2Amp, 
                       #'hddsn, pfcode, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, MRResistance, InputImpedance, SelectReaderGain, CalcedAeGain, AeGainCompensation_ResolutionAVG, AeGainCompensation_Amplitude1AVG, AeGainCompensation_Amplitude2AVG ')
                       'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, AeGainCompensation_ResolutionAVG as res, AeGainCompensation_Amplitude1AVG as amp2t, AeGainCompensation_Amplitude2AVG as amp10t, \
                        row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                       'band < 3'
                      )
dfAmp = pd.read_pickle('df_ccb_ci_amp.pkl')
dfAmpFlat = pd.pivot_table(dfAmp.query("band<3 and row_num==1"), 
                           index=['hddsn', 'mfgid', 'testcode', 'lhd', 'phd'], 
                           columns=['qualifier', 'band'], 
                           values=['res', 'amp2t', 'amp10t'], 
                           aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfAmpFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfAmpFlat.columns)[5:]]
dfAmpFlat.columns = cols
dfAmpFlat

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, AeGainCompensation_ResolutionAVG as res, AeGainCompensation_Amplitude1AVG as amp2t, AeGainCompensation_Amplitude2AVG as amp10t,                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_amp where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400', '6600', '6800') and qualifier in ('1000', '1010', '2000', '2010', '3000', '3010', '4000', '4010', '9000', '9010', 'K000', 'K010', 'B000', 'B010', 'M000', 'M010') and pfcode not in ('9999', '99V6', '99V7') and band < 3 
(2880, 13)


,hddsn,mfgid,testcode,lhd,phd,amp10t_1000_0,amp10t_1000_1,amp10t_1000_2,amp10t_1010_0,amp10t_1010_1,...,res_K000_2,res_K010_0,res_K010_1,res_K010_2,res_M000_0,res_M000_1,res_M000_2,res_M010_0,res_M010_1,res_M010_2
0,2FA00E8A,KJ1E03,PDQX083M,0,0,22.080827,23.114320,21.863354,12.209381,12.591803,...,45.593206,51.349424,40.603803,47.248244,51.773878,41.722708,45.540595,52.807315,40.840007,46.442703
1,2FA00E8A,KJ1E03,PDQX083M,1,1,34.558034,36.167616,37.050456,22.696140,23.193521,...,45.315435,45.072575,39.932332,45.313039,47.039322,39.273691,45.134477,45.030943,39.216231,45.755818
2,2FA00E8A,KJ1E03,PDQX083M,2,8,25.001595,25.759218,26.049515,15.249475,15.575805,...,41.512047,43.702547,37.236827,40.634333,47.056304,39.348751,42.375971,45.166553,37.864353,40.701511
3,2FA00E8A,KJ1E03,PDQX083M,3,9,36.054833,37.279027,37.037803,21.790889,22.460808,...,47.475679,49.280233,41.534868,47.813901,52.738328,45.017640,47.836641,49.517819,41.666679,47.280443
4,2FA00E8A,KJ1E03,PDQX083M,4,10,26.156166,27.561968,27.200210,19.311744,19.639282,...,40.810736,43.359496,34.724830,40.191867,49.282683,37.765034,41.400480,45.191099,34.502144,40.229196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2FA026TA,KJ1E03,PDQX083M,3,9,33.873992,34.350100,34.866252,26.916044,27.852416,...,47.551696,44.636353,37.810320,45.431287,49.136581,41.886673,47.739865,47.003717,39.465741,45.690562
60,2FA026TA,KJ1E03,PDQX083M,4,10,25.519600,26.539340,25.145537,28.569915,29.434602,...,46.301987,49.394299,41.041196,44.201891,55.526472,45.618030,46.261407,52.336740,41.991260,44.639068
61,2FA026TA,KJ1E03,PDQX083M,5,11,19.363646,19.629745,19.538446,27.738094,28.423276,...,43.111496,46.940813,37.634153,38.091395,49.702848,40.624674,42.787096,48.874026,39.243352,39.541233
62,2FA026TA,KJ1E03,PDQX083M,6,16,30.170148,30.430157,28.010968,33.823803,35.540478,...,51.036334,51.289435,42.884166,49.400996,59.360813,49.089122,51.758131,51.604340,42.674507,48.325254


In [23]:
## Asym ##

inputParamGhl2Asym = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400', '6600', '6800'],
                      ['1000', '1010', '2000', '2010',
                       '3000', '3010', '4000', '4010', 
                       '9000', '9010', 'K000', 'K010',
                       'B000', 'B010', 'M000', 'M010'
                      ]
                     ]
dfAsym = get_ghl2_table('ccb_ci_asm', 
                       inputParamGhl2Asym, 
                       'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, asymmetry, \
                        row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                       'band < 3'
                       )
"""
dfAsym = pd.read_pickle('df_ccb_ci_asm.pkl')
dfAsymFlat = pd.pivot_table(dfAsym.query("band<3 and row_num==1"), 
                           index=['hddsn', 'mfgid', 'testcode', 'lhd', 'phd'], 
                           columns=['qualifier', 'band'], 
                           values=['asymmetry'], 
                           aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfAsymFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfAsymFlat.columns)[5:]]
dfAsymFlat.columns = cols
dfAsymFlat
"""

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, asymmetry,                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_asm where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400', '6600', '6800') and qualifier in ('1000', '1010', '2000', '2010', '3000', '3010', '4000', '4010', '9000', '9010', 'K000', 'K010', 'B000', 'B010', 'M000', 'M010') and pfcode not in ('9999', '99V6', '99V7') and band < 3 
(2880, 11)


'\ndfAsym = pd.read_pickle(\'df_ccb_ci_asm.pkl\')\ndfAsymFlat = pd.pivot_table(dfAsym.query("band<3 and row_num==1"), \n                           index=[\'hddsn\', \'mfgid\', \'testcode\', \'lhd\', \'phd\'], \n                           columns=[\'qualifier\', \'band\'], \n                           values=[\'asymmetry\'], \n                           aggfunc=np.sum).reset_index(drop=False)\ncols = [x[0] for x in list(dfAsymFlat.columns)[:5]]+[str(x[0])+\'_\'+str(x[1])+\'_\'+str(x[2]) for x in list(dfAsymFlat.columns)[5:]]\ndfAsymFlat.columns = cols\ndfAsymFlat\n'

In [24]:
## RWIP ##

inputParamGhl2Rwip = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400', '6600', '6800'],
                      ['1000', '1010', '2000', '2010', 
                       '3000', '3010', '4000', '4010', 
                       '9000', '9010', 'K000', 'K010',
                       'B000', 'B010', 'M000', 'M010'
                      ]
                     ]
dfRwip = get_ghl2_table('ccb_ci_rwip', 
                        inputParamGhl2Rwip, 
                       'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, snrtotal, snrsystem, snrmedia, \
                        row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                        'band < 3'
                       )
"""
dfRwip = pd.read_pickle('df_ccb_ci_rwip.pkl').astype({'snrtotal': 'float64', 'snrsystem': 'float64', 'snrmedia': 'float64'})
dfRwipFlat = pd.pivot_table(dfRwip.query("band<3 and row_num==1"), 
                           index=['hddsn', 'mfgid', 'testcode', 'lhd', 'phd'], 
                           columns=['qualifier', 'band'], 
                           values=['snrtotal', 'snrsystem', 'snrmedia'], 
                           aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfRwipFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfRwipFlat.columns)[5:]]
dfRwipFlat.columns = cols
dfRwipFlat
"""

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, snrtotal, snrsystem, snrmedia,                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_rwip where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400', '6600', '6800') and qualifier in ('1000', '1010', '2000', '2010', '3000', '3010', '4000', '4010', '9000', '9010', 'K000', 'K010', 'B000', 'B010', 'M000', 'M010') and pfcode not in ('9999', '99V6', '99V7') and band < 3 
(2880, 13)


'\ndfRwip = pd.read_pickle(\'df_ccb_ci_rwip.pkl\').astype({\'snrtotal\': \'float64\', \'snrsystem\': \'float64\', \'snrmedia\': \'float64\'})\ndfRwipFlat = pd.pivot_table(dfRwip.query("band<3 and row_num==1"), \n                           index=[\'hddsn\', \'mfgid\', \'testcode\', \'lhd\', \'phd\'], \n                           columns=[\'qualifier\', \'band\'], \n                           values=[\'snrtotal\', \'snrsystem\', \'snrmedia\'], \n                           aggfunc=np.sum).reset_index(drop=False)\ncols = [x[0] for x in list(dfRwipFlat.columns)[:5]]+[str(x[0])+\'_\'+str(x[1])+\'_\'+str(x[2]) for x in list(dfRwipFlat.columns)[5:]]\ndfRwipFlat.columns = cols\ndfRwipFlat\n'

# AIw

In [25]:
#AIw
inputParamGhl2AIw = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400'],
                      ['10N0', '11N0', '30N0', '20N0', '40N0']
                     ]
dfAIw = get_ghl2_table('ccb_ci_aiw_summary', 
                        inputParamGhl2AIw, 
                       'hddsn, enddate, pfcode, testcode, mfgid, mfgid_5, hddtrial, qualifier, lhd, phd, readerselection, \
                        cmdexecutiontimeinmilliseconds as time, band, cylinder, meastemp, \
                        writecurrent as iw, kickamplitude as ka, kickduration as kd, bestser, readoffset, enddt, \
                        row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num'
                       )
dfAIw.loc[:,'mfgid_5'] = dfAIw['mfgid_5'].values.astype('int')
#Gen4
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_a'] = -0.013954886
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_b'] = 1.128161219
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_c'] = 1.973010793
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_d'] = -1.407847328
#Gen4i A1
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_a'] = -0.015470889
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_b'] = 1.684161818
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_c'] = 1.335027916
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_d'] = 0.387637386

dfAIw.loc[:,'ipk'] = ((dfAIw.loc[:,'coef_a'].values * dfAIw.loc[:,'ka'].values + dfAIw.loc[:,'coef_b'].values) * dfAIw.loc[:,'iw'].values +
                      (dfAIw.loc[:,'coef_c'].values * dfAIw.loc[:,'ka'].values + dfAIw.loc[:,'coef_d'].values)
                     )
dfAIw.to_pickle('dfAIw.pkl')
dfAIw

current sql is select hddsn, enddate, pfcode, testcode, mfgid, mfgid_5, hddtrial, qualifier, lhd, phd, readerselection,                         cmdexecutiontimeinmilliseconds as time, band, cylinder, meastemp,                         writecurrent as iw, kickamplitude as ka, kickduration as kd, bestser, readoffset, enddt,                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_aiw_summary where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('10N0', '11N0', '30N0', '20N0', '40N0') and pfcode not in ('9999', '99V6', '99V7') 
(2304, 22)


,hddsn,enddate,pfcode,testcode,mfgid,mfgid_5,hddtrial,qualifier,lhd,phd,...,kd,bestser,readoffset,enddt,row_num,coef_a,coef_b,coef_c,coef_d,ipk
0,2FA00EPA,20201228063812,0000,PDQX083M,KJ1E03,3,Z07E,40N0,3,9,...,4,-2.10,0.00,20201228,1,-0.013955,1.128161,1.973011,-1.407847,84.744033
1,2FA00EWA,20201228080909,0000,PDQX083M,KJ1E03,3,Z07E,10N0,4,10,...,4,-3.08,-1.32,20201228,1,-0.013955,1.128161,1.973011,-1.407847,87.577877
2,2FA00EYA,20201228083039,0000,PDQX083M,KJ1E03,3,Z07E,10N0,3,9,...,4,-3.09,-0.17,20201228,1,-0.013955,1.128161,1.973011,-1.407847,86.882318
3,2FA00EPA,20201228063812,0000,PDQX083M,KJ1E03,3,Z07E,10N0,2,8,...,4,-3.09,-0.25,20201228,1,-0.013955,1.128161,1.973011,-1.407847,86.882318
4,2FA00EPA,20201228063812,0000,PDQX083M,KJ1E03,3,Z07E,30N0,7,17,...,4,-2.12,-0.64,20201228,1,-0.013955,1.128161,1.973011,-1.407847,86.882318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,2FA00E8A,20201228075844,0000,PDQX083M,KJ1E03,3,Z07E,11N0,5,11,...,4,-1.69,-0.74,20201228,1,-0.013955,1.128161,1.973011,-1.407847,82.577838
2300,2FA01HYA,20201228072658,0000,PDQX083M,KJBE03,3,Z07E,20N0,7,17,...,4,-3.13,5.40,20201228,1,-0.013955,1.128161,1.973011,-1.407847,86.882318
2301,2FA00EPA,20201228063812,0000,PDQX083M,KJ1E03,3,Z07E,10N0,1,1,...,4,-3.07,-0.07,20201228,1,-0.013955,1.128161,1.973011,-1.407847,89.674298
2302,2FA026TA,20201228075939,0000,PDQX083M,KJ1E03,3,Z07E,10N0,0,0,...,4,-2.67,-0.47,20201228,1,-0.013955,1.128161,1.973011,-1.407847,84.744033


In [26]:
#AIw Profile
inputParamGhl2AIw = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400'],
                      ['10N0', '11N0', '30N0', '20N0', '40N0']
                     ]
dfAIw = get_ghl2_table('ccb_ci_aiw_prim_comb', 
                        inputParamGhl2AIw, 
                       'hddsn, enddate, pfcode, testcode, mfgid, mfgid_5, hddtrial, qualifier, lhd, phd, readerselection, \
                        band, index, writecurlist as iw, kickamp as ka, peakiw as ipk0, servswrparm as ser, \
                        row_number() over ( partition by hddsn, qualifier, lhd, band, peakiw, testcode order by enddate desc) as row_num'
                       )
dfAIw.loc[:,'mfgid_5'] = dfAIw['mfgid_5'].values.astype('int')

#Gen4
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_a'] = -0.013954886
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_b'] = 1.128161219
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_c'] = 1.973010793
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_d'] = -1.407847328
#Gen4i A1
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_a'] = -0.015470889
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_b'] = 1.684161818
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_c'] = 1.335027916
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_d'] = 0.387637386

dfAIw.loc[:,'ipk'] = ((dfAIw.loc[:,'coef_a'].values * dfAIw.loc[:,'ka'].values + dfAIw.loc[:,'coef_b'].values) * dfAIw.loc[:,'iw'].values +
                      (dfAIw.loc[:,'coef_c'].values * dfAIw.loc[:,'ka'].values + dfAIw.loc[:,'coef_d'].values)
                     )
dfAIw.to_pickle('dfAIwProfile.pkl')
dfAIw

current sql is select hddsn, enddate, pfcode, testcode, mfgid, mfgid_5, hddtrial, qualifier, lhd, phd, readerselection,                         band, index, writecurlist as iw, kickamp as ka, peakiw as ipk0, servswrparm as ser,                         row_number() over ( partition by hddsn, qualifier, lhd, band, peakiw, testcode order by enddate desc) as row_num from ghl2.ccb_ci_aiw_prim_comb where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('10N0', '11N0', '30N0', '20N0', '40N0') and pfcode not in ('9999', '99V6', '99V7') 
(59904, 18)


,hddsn,enddate,pfcode,testcode,mfgid,mfgid_5,hddtrial,qualifier,lhd,phd,...,iw,ka,ipk0,ser,row_num,coef_a,coef_b,coef_c,coef_d,ipk
0,2FA00E8A,20201228075844,0000,PDQX083M,KJ1E03,3,Z07E,10N0,2,8,...,46,37,100,-3.21,1,-0.013955,1.128161,1.973011,-1.407847,99.737752
1,2FA00E8A,20201228075844,0000,PDQX083M,KJ1E03,3,Z07E,10N0,3,9,...,58,53,126,-2.95,1,-0.013955,1.128161,1.973011,-1.407847,125.697756
2,2FA00E8A,20201228075844,0000,PDQX083M,KJ1E03,3,Z07E,11N0,3,9,...,38,30,85,-1.66,1,-0.013955,1.128161,1.973011,-1.407847,84.744033
3,2FA00E8A,20201228075844,0000,PDQX083M,KJ1E03,3,Z07E,20N0,2,8,...,42,32,90,-2.83,1,-0.013955,1.128161,1.973011,-1.407847,90.355902
4,2FA00E8A,20201228075844,0000,PDQX083M,KJ1E03,3,Z07E,30N0,4,10,...,52,44,112,-2.09,1,-0.013955,1.128161,1.973011,-1.407847,112.140232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59899,2FA026TA,20201228075939,0000,PDQX083M,KJ1E03,3,Z07E,20N0,3,9,...,36,28,80,-3.07,1,-0.013955,1.128161,1.973011,-1.407847,80.383734
59900,2FA026TA,20201228075939,0000,PDQX083M,KJ1E03,3,Z07E,30N0,1,1,...,32,24,71,-1.89,1,-0.013955,1.128161,1.973011,-1.407847,71.328218
59901,2FA026TA,20201228075939,0000,PDQX083M,KJ1E03,3,Z07E,30N0,2,8,...,36,28,80,-2.15,1,-0.013955,1.128161,1.973011,-1.407847,80.383734
59902,2FA026TA,20201228075939,0000,PDQX083M,KJ1E03,3,Z07E,30N0,3,9,...,38,30,85,-2.04,1,-0.013955,1.128161,1.973011,-1.407847,84.744033


# ATI

In [27]:
#DATI Nmax
inputParamGhl2Dati = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6400', '6600'],
                     ['3000', '9000', '4360', 'L360']
                    ]
dfNmaxTemp = get_ghl2_table('ccb_ci_ati_prim_bybandwrnum', 
                            inputParamGhl2Dati, 
                            'hddsn, enddt, mfgid, hddtrial, testcode, qualifier, procid, lhd, phd, band, nmax50byband, \
                             row_number() over (partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num')
dfNmaxTemp = pd.read_pickle('df_ccb_ci_ati_prim_bybandwrnum.pkl')
dfNmaxTemp.loc[:,'nmax50byband'] = dfNmaxTemp.loc[:,'nmax50byband'].values.astype(np.int64)
dfNmaxTemp.to_pickle('dfNmaxTemp.pkl')
dfNmaxTemp

current sql is select hddsn, enddt, mfgid, hddtrial, testcode, qualifier, procid, lhd, phd, band, nmax50byband,                              row_number() over (partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_ati_prim_bybandwrnum where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400', '6600') and qualifier in ('3000', '9000', '4360', 'L360') and pfcode not in ('9999', '99V6', '99V7') 
(8192, 12)


,hddsn,enddt,mfgid,hddtrial,testcode,qualifier,procid,lhd,phd,band,nmax50byband,row_num
0,2FA00E8A,20201228,KJ1E03,Z07E,PDQX083M,3000,6400,2,8,47,579,1
1,2FA00EPA,20201228,KJ1E03,Z07E,PDQX083M,3000,6400,6,16,53,550,1
2,2FA00ESA,20210105,KJ1E03,Z07E,PDQX083M,9000,6600,1,1,0,551,1
3,2FA00EYA,20201228,KJ1E03,Z07E,PDQX083M,3000,6400,4,10,58,2572,1
4,2FA00EYA,20201228,KJ1E03,Z07E,PDQX083M,3000,6400,5,11,28,718,1
...,...,...,...,...,...,...,...,...,...,...,...,...
8187,2FA00EYA,20201228,KJ1E03,Z07E,PDQX083M,3000,6400,0,0,26,665,1
8188,2FA00EYA,20201228,KJ1E03,Z07E,PDQX083M,3000,6400,5,11,57,812,1
8189,2FA00EYA,20210105,KJ1E03,Z07E,PDQX083M,9000,6600,1,1,47,248,1
8190,2FA01HYA,20210105,KJBE03,Z07E,PDQX083M,9000,6600,1,1,59,458,1


In [28]:
#DATI RV
inputParamGhl2AtiRv = [snlist_csv, 
                       listTestcode,
                       listEnddt, 
                       listHddTrial,
                       ['6400', '6600'],
                       ['35N0', '3000', '95H0', '9000', '4360', 'L360']
                      ]

dfAtiRv = get_ghl2_table('ccb_ci_dati_theta', 
                         inputParamGhl2AtiRv, 
                         'hddsn, enddate, pfcode, testcode, testcodec, mfgid, hddtrial, cmdname, qualifier, lhd, phd, subqualifier, readerselection, tempzone, zone, \
                          maxrvsumheadalltemp, avgrvsumheadalltemp, avgrvsumdrivealltemp, avgrvsumzonealltemp, rvsum, spaidentifier, rvsumminushalf4smr, rvsumplushalf4smr, \
                          worstavgrvsumzonewithmaxnlxmulti, worstzonewithmaxnlxmulti, worstavgrvsumzonewithminnlxmulti, worstzonewithminnlxmulti, worstavgrvsumzonewithavenlxmulti, \
                          worstzonewithavenlxmulti, product, procid, enddt, \
                          row_number() over ( partition by hddsn, testcode, qualifier, lhd, tempzone, zone order by enddate desc) as row_num')
#dfAtiRv = pd.read_pickle('df_ccb_ci_dati_theta.pkl')
dfAtiRv

current sql is select hddsn, enddate, pfcode, testcode, testcodec, mfgid, hddtrial, cmdname, qualifier, lhd, phd, subqualifier, readerselection, tempzone, zone,                           maxrvsumheadalltemp, avgrvsumheadalltemp, avgrvsumdrivealltemp, avgrvsumzonealltemp, rvsum, spaidentifier, rvsumminushalf4smr, rvsumplushalf4smr,                           worstavgrvsumzonewithmaxnlxmulti, worstzonewithmaxnlxmulti, worstavgrvsumzonewithminnlxmulti, worstzonewithminnlxmulti, worstavgrvsumzonewithavenlxmulti,                           worstzonewithavenlxmulti, product, procid, enddt,                           row_number() over ( partition by hddsn, testcode, qualifier, lhd, tempzone, zone order by enddate desc) as row_num from ghl2.ccb_ci_dati_theta where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in (

,hddsn,enddate,pfcode,testcode,testcodec,mfgid,hddtrial,cmdname,qualifier,lhd,...,worstavgrvsumzonewithmaxnlxmulti,worstzonewithmaxnlxmulti,worstavgrvsumzonewithminnlxmulti,worstzonewithminnlxmulti,worstavgrvsumzonewithavenlxmulti,worstzonewithavenlxmulti,product,procid,enddt,row_num
0,2FA00E8A,20210105051406,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9000,2,...,0,0,0,0,0,0,pdq,6600,20210105,1
1,2FA00E8A,20210105051406,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9000,2,...,0,0,0,0,0,0,pdq,6600,20210105,1
2,2FA00E8A,20210105051406,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9000,5,...,0,0,0,0,0,0,pdq,6600,20210105,1
3,2FA00E8A,20210105051406,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9000,5,...,0,0,0,0,0,0,pdq,6600,20210105,1
4,2FA00EPA,20201228063812,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,3000,1,...,0,0,0,0,0,0,pdq,6400,20201228,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35323,2FA026TA,20201228075939,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,3000,6,...,0,0,0,0,0,0,pdq,6400,20201228,1
35324,2FA026TA,20201228075939,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,3000,7,...,0,0,0,0,0,0,pdq,6400,20201228,1
35325,2FA026TA,20210105131116,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9000,1,...,0,0,0,0,0,0,pdq,6600,20210105,1
35326,2FA026TA,20210105131116,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9000,2,...,0,0,0,0,0,0,pdq,6600,20210105,1


In [29]:
#JINX RV
inputParamGhl2AtiRv = [snlist_csv, 
                       listTestcode,
                       listEnddt, 
                       listHddTrial,
                       ['6600'],
                       ['9050']
                      ]

dfJinxRv = get_ghl2_table('ccb_ci_ati_mcsb_ns', 
                         inputParamGhl2AtiRv, 
                         'hddsn, enddate, pfcode, testcode, testcodec, mfgid, hddtrial, cmdname, qualifier, lhd, phd, maxrvcheck4nonlinearxcor_0, maxrvcheck4nonlinearxcor_1, \
                          row_number() over ( partition by hddsn, testcode, qualifier, lhd order by enddate desc) as row_num')
dfJinxRv

current sql is select hddsn, enddate, pfcode, testcode, testcodec, mfgid, hddtrial, cmdname, qualifier, lhd, phd, maxrvcheck4nonlinearxcor_0, maxrvcheck4nonlinearxcor_1,                           row_number() over ( partition by hddsn, testcode, qualifier, lhd order by enddate desc) as row_num from ghl2.ccb_ci_ati_mcsb_ns where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6600') and qualifier in ('9050') and pfcode not in ('9999', '99V6', '99V7') 
(64, 14)


,hddsn,enddate,pfcode,testcode,testcodec,mfgid,hddtrial,cmdname,qualifier,lhd,phd,maxrvcheck4nonlinearxcor_0,maxrvcheck4nonlinearxcor_1,row_num
0,2FA00EPA,20210105071529,TSW1,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9050,4,10,1380,15000,1
1,2FA00EWA,20210105063442,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9050,2,8,1950,15000,1
2,2FA00EPA,20210105071529,TSW1,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9050,6,16,1700,15000,1
3,2FA00ESA,20210105045411,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9050,2,8,1461,15000,1
4,2FA01HYA,20210105062412,0000,PDQX083M,PDQX083,KJBE03,Z07E,CAti,9050,5,11,1838,15000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2FA01HYA,20210105062412,0000,PDQX083M,PDQX083,KJBE03,Z07E,CAti,9050,0,0,3141,15000,1
60,2FA00EYA,20210105062246,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9050,3,9,1126,15000,1
61,2FA00EPA,20210105071529,TSW1,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9050,0,0,1132,15000,1
62,2FA026TA,20210105131116,0000,PDQX083M,PDQX083,KJ1E03,Z07E,CAti,9050,5,11,1096,15000,1


# ASB (SMR)

In [30]:
#CASB
inputParamGhl2Asb = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6400'],
                     ['2300']
                    ]
dfAsb = get_ghl2_table('ccb_ci_casb', inputParamGhl2Asb, 
                       'hddsn, enddate, lhd, phd, mfgid, testcode, maxbandod2id, \
                        row_number() over (partition by hddsn, testcode, lhd, band order by enddate desc) as row_num', 
                       'band=0')
dfAsb = pd.read_pickle('df_ccb_ci_casb.pkl').query("row_num==1")
dfAsb

current sql is select hddsn, enddate, lhd, phd, mfgid, testcode, maxbandod2id,                         row_number() over (partition by hddsn, testcode, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_casb where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('2300') and pfcode not in ('9999', '99V6', '99V7') and band=0 
(64, 8)


,hddsn,enddate,lhd,phd,mfgid,testcode,maxbandod2id,row_num
0,2FA026TA,20201228075939,2,8,KJ1E03,PDQX083M,29,1
1,2FA01HYA,20201228072658,2,8,KJBE03,PDQX083M,31,1
2,2FA00EYA,20201228083039,0,0,KJ1E03,PDQX083M,48,1
3,2FA01HYA,20201228072658,4,10,KJBE03,PDQX083M,35,1
4,2FA00E8A,20201228075844,6,16,KJ1E03,PDQX083M,32,1
...,...,...,...,...,...,...,...,...
59,2FA00EPA,20201228063812,0,0,KJ1E03,PDQX083M,29,1
60,2FA00E8A,20201228075844,7,17,KJ1E03,PDQX083M,42,1
61,2FA00EPA,20201228063812,1,1,KJ1E03,PDQX083M,32,1
62,2FA026TA,20201228075939,1,1,KJ1E03,PDQX083M,61,1


# UFO

In [31]:
# UFO
inputParamGhl2Ufo = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6400'],
                     ['1000', '2000']
                    ]
dfUfo = get_ghl2_table('ccb_ci_ufo', inputParamGhl2Ufo, 
                       'hddsnenddate, hddsn, enddate, drivemodel, pfcode, testcode, mfgid, hddtrial, aetype, hdctype, cmdname, \
                       qualifier, readerselection, cmdexecutiontimeinmilliseconds, mcsbarraymaxnumheads, mcsbarraymaxnumbands, \
                       mcsbarrayodmdidbands, logicalheadtestcompletemap, physicalheadfailcriteriamap, lastlogicalheadtested, lastbandtested, \
                       lasttracktested, lastsubcmdtested, finalusedservotrack, finalstrokepctx100, finalstrokepercent, finaldefectupliftpctx100, \
                       finalmediacacheupliftpctx100, finaltrackskewadjustment, finaltpirelaxationpercentx100, finaltpirelaxationpercent, \
                       finalwaterfallselection, finalblocksizemap, finalfitmetricvalue, finalmaxlogicalheadnum, finalmodelnumber, finalphysicalheadmap, \
                       finalsustainedmbytepersec, totaldrivelbarequired, finaldrivetotallba, calcsteps, accdrivetotallba, accdrivetotallbanoati, \
                       maxchanfreqmhz, accdrivetotallbaformat, finalseqperfvarpctx100, accdefaultupliftcalcreqcap, product, procid, enddt, \
                       row_number() over (partition by hddsn, testcode, qualifier order by enddate desc) as row_num')
#dfUfo.loc[dfUfo.qualifier.str.startswith('1'), 'targetLba'] = tgtLbaCmr
#dfUfo.loc[dfUfo.qualifier.str.startswith('2'), 'targetLba'] = tgtLbaSmr
#dfUfo.loc[:,'iacc_pct'] = dfUfo.loc[:,'iacc_pct'].values / tgtLbaCmr
dfUfo = pd.read_pickle('df_ccb_ci_ufo.pkl')

current sql is select hddsnenddate, hddsn, enddate, drivemodel, pfcode, testcode, mfgid, hddtrial, aetype, hdctype, cmdname,                        qualifier, readerselection, cmdexecutiontimeinmilliseconds, mcsbarraymaxnumheads, mcsbarraymaxnumbands,                        mcsbarrayodmdidbands, logicalheadtestcompletemap, physicalheadfailcriteriamap, lastlogicalheadtested, lastbandtested,                        lasttracktested, lastsubcmdtested, finalusedservotrack, finalstrokepctx100, finalstrokepercent, finaldefectupliftpctx100,                        finalmediacacheupliftpctx100, finaltrackskewadjustment, finaltpirelaxationpercentx100, finaltpirelaxationpercent,                        finalwaterfallselection, finalblocksizemap, finalfitmetricvalue, finalmaxlogicalheadnum, finalmodelnumber, finalphysicalheadmap,                        finalsustainedmbytepersec, totaldrivelbarequired, finaldrivetotallba, calcsteps, accdrivetotallba, accdrivetotallbanoati,                        maxch

In [32]:
# UFO hd array
inputParamGhl2UfoHdArry = [snlist_csv, 
                           listTestcode,
                           listEnddt, 
                           listHddTrial,
                           ['6400'],
                           ['1000', '2000']
                    ]
dfUfoHdArry = get_ghl2_table('ccb_ci_ufo_hd_array', inputParamGhl2UfoHdArry, 
                             'hddsn, enddate, pfcode, testcode, mfgid, hddtrial, qualifier, lhd, readerselection, \
                             finalheadtotallba, accheadtotallba, accmetricvalue, accheadtotallbanoati, \
                             accheadtotallbaformat, \
                             row_number() over (partition by hddsn, lhd, testcode, qualifier order by enddate desc) as row_num',
                            )
dfUfoHdArry.loc[dfUfoHdArry.qualifier.str.startswith('1'), 'targetLba'] = tgtLbaCmr
dfUfoHdArry.loc[dfUfoHdArry.qualifier.str.startswith('2'), 'targetLba'] = tgtLbaSmr
dfUfoHdArry.loc[:,'iacc_pct'] = 100*dfUfoHdArry.loc[:,'accheadtotallbanoati'].values  / dfUfoHdArry.loc[:, 'targetLba'].values
dfUfoHdArry.loc[:,'pacc_pct'] = 100*dfUfoHdArry.loc[:,'accheadtotallbaformat'].values / dfUfoHdArry.loc[:, 'targetLba'].values
dfUfoHdArry.to_pickle('dfUfoHdArry.pkl')
dfUfoHdArry

current sql is select hddsn, enddate, pfcode, testcode, mfgid, hddtrial, qualifier, lhd, readerselection,                              finalheadtotallba, accheadtotallba, accmetricvalue, accheadtotallbanoati,                              accheadtotallbaformat,                              row_number() over (partition by hddsn, lhd, testcode, qualifier order by enddate desc) as row_num from ghl2.ccb_ci_ufo_hd_array where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('1000', '2000') and pfcode not in ('9999', '99V6', '99V7') 
(128, 15)


,hddsn,enddate,pfcode,testcode,mfgid,hddtrial,qualifier,lhd,readerselection,finalheadtotallba,accheadtotallba,accmetricvalue,accheadtotallbanoati,accheadtotallbaformat,row_num,targetLba,iacc_pct,pacc_pct
0,2FA01K0A,20201228083006,0000,PDQX083M,KJBE03,Z07E,1000,0,3951,248631742,None,None,256608110,254422454,1,273906721.0,93.684488,92.886532
1,2FA00EWA,20201228080909,0000,PDQX083M,KJ1E03,Z07E,1000,3,3951,254807434,None,None,263497914,262775261,1,273906721.0,96.199872,95.936040
2,2FA026TA,20201228075939,0000,PDQX083M,KJ1E03,Z07E,1000,5,3951,245924770,None,None,252804093,252958908,1,273906721.0,92.295688,92.352209
3,2FA01K0A,20201228083006,0000,PDQX083M,KJBE03,Z07E,1000,1,3951,243963607,None,None,254466420,252897877,1,273906721.0,92.902583,92.329928
4,2FA00ESA,20201228073006,0000,PDQX083M,KJ1E03,Z07E,1000,5,4079,248844239,None,None,258498622,256813679,1,273906721.0,94.374691,93.759539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2FA00ESA,20201228073006,0000,PDQX083M,KJ1E03,Z07E,2000,6,4079,277809549,None,None,285205977,285205977,1,333662709.0,85.477331,85.477331
124,2FA00EPA,20201228063812,0000,PDQX083M,KJ1E03,Z07E,2000,5,4079,263689107,None,None,278974559,278974559,1,333662709.0,83.609751,83.609751
125,2FA00EWA,20201228080909,0000,PDQX083M,KJ1E03,Z07E,1000,6,3951,247461371,None,None,256101084,255684267,1,273906721.0,93.499379,93.347205
126,2FA00E8A,20201228075844,0000,PDQX083M,KJ1E03,Z07E,2000,1,3951,283188463,None,None,293637056,293637056,1,333662709.0,88.004158,88.004158


In [33]:
pd.pivot_table(dfUfoHdArry.query("qualifier=='1000'"), index=['hddsn', 'mfgid', 'lhd'], columns=['testcode'], values=['pacc_pct'], 
               aggfunc=np.mean)

pacc_pct
testcode              PDQX083M
hddsn    mfgid  lhd           
2FA00E8A KJ1E03 0    94.173948
                1    94.427803
                2    90.965912
                3    94.136842
                4    94.467430
...                        ...
2FA026TA KJ1E03 3    94.771224
                4    94.992711
                5    92.352209
                6    95.632536
                7    92.847835

[64 rows x 1 columns]

In [34]:
# UFO SER, TPC
inputParamGhl2UfoSubCmd1 = [snlist_csv, 
                            listTestcode,
                            listEnddt, 
                            listHddTrial,
                            ['6400'],
                            ['1000', '2000']
                           ]
dfUfoSubCmd1 = get_ghl2_table('ccb_ci_ufo_scmd1', inputParamGhl2UfoSubCmd1, 
                              '* ')
dfUfoSubCmd1 = pd.read_pickle('df_ccb_ci_ufo_scmd1.pkl')

current sql is select *  from ghl2.ccb_ci_ufo_scmd1 where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('1000', '2000') and pfcode not in ('9999', '99V6', '99V7') 
(16493, 62)


In [35]:
# UFO Sqz Margin, otrc, track pitch
inputParamGhl2UfoSubCmd7 = [snlist_csv, 
                            listTestcode,
                            listEnddt, 
                            listHddTrial,
                            ['6400', '6600'],
                            ['3000', '9000', '4000', 'K000']
                           ]
dfUfoSubCmd7 = get_ghl2_table('ccb_ci_ufo_scmd7', 
                              inputParamGhl2UfoSubCmd7, 
                              'hddsn, enddate, testcode, mfgid, hddtrial, qualifier, lhd, test_band, tpmtpcdbl, tpmtpcod, tpmtpcid, otrc, finkfci, finktpi, defaultrate, procid, enddt, \
                               row_number() over (partition by hddsn, testcode, qualifier, lhd, test_band order by enddate desc) as row_num')
#dfUfoSubCmd7 = pd.read_pickle('df_ccb_ci_ufo_scmd7.pkl')

dfUfoSubCmd7.query("qualifier in ['3000', '9000', '4000', 'K000']", inplace=True)
dfUfoSubCmd7.loc[:,'tp_nm']  = (dfUfoSubCmd7.loc[:,'defaultrate'].values / 4096.0 * 25.4/stpi * 1e3).astype(np.float64)
if dfUfoSubCmd7.query("qualifier in ['3000', '9000']").shape[0] > 0:
    dfUfoSubCmd7.loc[(dfUfoSubCmd7['qualifier']=='3000') | (dfUfoSubCmd7['qualifier']=='9000'),'dtpi'] = dtpi_cmr
if dfUfoSubCmd7.query("qualifier in ['4000', 'K000']").shape[0] > 0:    
    dfUfoSubCmd7.loc[(dfUfoSubCmd7['qualifier']=='4000') | (dfUfoSubCmd7['qualifier']=='K000'),'dtpi'] = dtpi_smr

dfUfoSubCmd7.loc[:,'otrc'] = (dfUfoSubCmd7.loc[:,'otrc'].values * 25.4/dfUfoSubCmd7.loc[:,'dtpi'].values).astype(np.float64)     
dfUfoSubCmd7.loc[:,'tpf_dbl'] = (dfUfoSubCmd7.loc[:,'tpmtpcdbl'].values / 10000.0 * 25.4/dfUfoSubCmd7.loc[:,'dtpi'].values * 1e3).astype(np.float64)
dfUfoSubCmd7.loc[:,'tpf_od'] = (dfUfoSubCmd7.loc[:,'tpmtpcod'].values / 10000.0 * 25.4/dfUfoSubCmd7.loc[:,'dtpi'].values * 1e3).astype(np.float64)
dfUfoSubCmd7.loc[:,'tpf_id'] = (dfUfoSubCmd7.loc[:,'tpmtpcid'].values / 10000.0 * 25.4/dfUfoSubCmd7.loc[:,'dtpi'].values * 1e3).astype(np.float64)

#valid calc for only SMR
dfUfoSubCmd7.loc[:,'tpf_smr'] = (dfUfoSubCmd7.loc[:,'tpf_od'].values + dfUfoSubCmd7.loc[:,'tpf_id']).astype(np.float64)
dfUfoSubCmd7.loc[:,'sqm_smr'] = (dfUfoSubCmd7.loc[:,'tp_nm'].values - dfUfoSubCmd7.loc[:,'tpf_smr'].values).astype(np.float64)                            

#valid calc for only CMR
dfUfoSubCmd7.loc[:,'sqm_dbl'] = (dfUfoSubCmd7.loc[:,'tp_nm'].values - dfUfoSubCmd7.loc[:,'tpf_dbl'].values).astype(np.float64)
dfUfoSubCmd7.loc[:,'sqm_od']  = (dfUfoSubCmd7.loc[:,'tp_nm'].values - dfUfoSubCmd7.loc[:,'tpf_od'].values).astype(np.float64)
dfUfoSubCmd7.loc[:,'sqm_id']  = (dfUfoSubCmd7.loc[:,'tp_nm'].values - dfUfoSubCmd7.loc[:,'tpf_id'].values).astype(np.float64)

dfUfoSubCmd7.to_pickle('dfUfoSubCmd7.pkl')

"""
dfSqmFlat = pd.pivot_table(dfUfoSubCmd7.query("row_num==1"), 
                               index=['hddsn', 'mfgid', 'hddtrial', 'testcode', 'lhd'], 
                               columns=['qualifier', 'test_band'], 
                               values=['tp_nm', 'tpf_dbl', 'tpf_od', 'tpf_id', 'tpf_smr', 'sqm_dbl', 'sqm_od', 'sqm_id', 'sqm_smr', 'otrc', 'finkfci', 'finktpi'], 
                               aggfunc=np.sum).reset_index(drop=False)
        
cols = [x[0] for x in list(dfSqmFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfSqmFlat.columns)[5:]]
dfSqmFlat.columns = cols
del dfUfoSubCmd7, cols
print('dfSqmFlat')
"""
dfUfoSubCmd7

current sql is select hddsn, enddate, testcode, mfgid, hddtrial, qualifier, lhd, test_band, tpmtpcdbl, tpmtpcod, tpmtpcid, otrc, finkfci, finktpi, defaultrate, procid, enddt,                                row_number() over (partition by hddsn, testcode, qualifier, lhd, test_band order by enddate desc) as row_num from ghl2.ccb_ci_ufo_scmd7 where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400', '6600') and qualifier in ('3000', '9000', '4000', 'K000') and pfcode not in ('9999', '99V6', '99V7') 
(16384, 18)


,hddsn,enddate,testcode,mfgid,hddtrial,qualifier,lhd,test_band,tpmtpcdbl,tpmtpcod,...,tp_nm,dtpi,tpf_dbl,tpf_od,tpf_id,tpf_smr,sqm_smr,sqm_dbl,sqm_od,sqm_id
0,2FA00E8A,20201228075844,PDQX083M,KJ1E03,Z07E,3000,3,54,9219,7173,...,54.014345,500.0,46.83252,36.438840,36.824920,73.263760,-19.249415,7.181825,17.575505,17.189425
1,2FA00E8A,20201228075844,PDQX083M,KJ1E03,Z07E,4000,3,23,0,0,...,42.638402,602.0,0.00000,0.000000,31.648738,31.648738,10.989665,42.638402,42.638402,10.989665
2,2FA00E8A,20201228075844,PDQX083M,KJ1E03,Z07E,4000,6,2,0,0,...,43.108836,602.0,0.00000,0.000000,32.290066,32.290066,10.818770,43.108836,43.108836,10.818770
3,2FA00E8A,20210105051406,PDQX083M,KJ1E03,Z07E,K000,4,11,0,0,...,44.135237,602.0,0.00000,0.000000,33.703522,33.703522,10.431715,44.135237,44.135237,10.431715
4,2FA00EPA,20201228063812,PDQX083M,KJ1E03,Z07E,4000,1,40,0,7927,...,43.878637,602.0,0.00000,33.446146,0.000000,33.446146,10.432491,43.878637,10.432491,43.878637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16379,2FA026TA,20201228075939,PDQX083M,KJ1E03,Z07E,4000,6,25,0,0,...,43.066072,602.0,0.00000,0.000000,31.733123,31.733123,11.332949,43.066072,43.066072,11.332949
16380,2FA026TA,20210105131116,PDQX083M,KJ1E03,Z07E,9000,0,46,8520,7013,...,49.609375,500.0,43.28160,35.626040,35.753040,71.379080,-21.769705,6.327775,13.983335,13.856335
16381,2FA026TA,20210105131116,PDQX083M,KJ1E03,Z07E,9000,1,19,8907,7594,...,52.731344,500.0,45.24756,38.577520,36.311840,74.889360,-22.158016,7.483784,14.153824,16.419504
16382,2FA026TA,20210105131116,PDQX083M,KJ1E03,Z07E,K000,2,4,0,0,...,42.681169,602.0,0.00000,0.000000,31.948306,31.948306,10.732864,42.681169,42.681169,10.732864


In [36]:
# UFO ADC, BPI, TPI
inputParamGhl2UfoHdBandArray = [snlist_csv, 
                                listTestcode,
                                listEnddt, 
                                listHddTrial,
                                ['6400'],
                                ['1000', '2000']
                               ]
dfUfoHdBandArray = get_ghl2_table('ccb_ci_ufo_hd_band_array', inputParamGhl2UfoHdBandArray, 
                                  'hddsn, enddt, pfcode, testcode, mfgid, hddtrial, qualifier, lhd, band, adc, adckbpi, adcktpi, \
                                   row_number() over (partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num')
dfUfoHdBandArray = pd.read_pickle('df_ccb_ci_ufo_hd_band_array.pkl')
dfUfoHdBandArray.loc[:,'adc']     = dfUfoHdBandArray.loc[:,'adc'].values.astype(np.float64)
dfUfoHdBandArray.loc[:,'adckbpi'] = dfUfoHdBandArray.loc[:,'adckbpi'].values.astype(np.float64)
dfUfoHdBandArray.loc[:,'adcktpi'] = dfUfoHdBandArray.loc[:,'adcktpi'].values.astype(np.float64)
dfUfoHdBandArray.to_pickle('dfUfoHdBandArray.pkl')

"""
dfAdcFlat = pd.pivot_table(dfUfoHdBandArray.query("band<7 and row_num==1"), 
                          index=['hddsn', 'mfgid', 'hddtrial', 'testcode', 'lhd'], 
                          columns=['qualifier', 'band'], 
                          values=['adc', 'adckbpi', 'adcktpi'], 
                          aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfAdcFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfAdcFlat.columns)[5:]]
dfAdcFlat.columns = cols
del dfUfoHdBandArray, cols
print('dfAdcFlat')
"""
dfUfoHdBandArray

current sql is select hddsn, enddt, pfcode, testcode, mfgid, hddtrial, qualifier, lhd, band, adc, adckbpi, adcktpi,                                    row_number() over (partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_ufo_hd_band_array where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('1000', '2000') and pfcode not in ('9999', '99V6', '99V7') 
(791, 13)


,hddsn,enddt,pfcode,testcode,mfgid,hddtrial,qualifier,lhd,band,adc,adckbpi,adcktpi,row_num
0,2FA026TA,20201228,0000,PDQX083M,KJ1E03,Z07E,1000,7,3,974.0,2223.0,438.0,1
1,2FA00E8A,20201228,0000,PDQX083M,KJ1E03,Z07E,1000,0,6,1055.0,2192.0,481.0,1
2,2FA00ESA,20201228,0000,PDQX083M,KJ1E03,Z07E,1000,1,6,1074.0,2122.0,506.0,1
3,2FA00E8A,20201228,0000,PDQX083M,KJ1E03,Z07E,1000,7,3,945.0,2009.0,470.0,1
4,2FA00EPA,20201228,0000,PDQX083M,KJ1E03,Z07E,1000,5,6,1058.0,2054.0,515.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,2FA00E8A,20201228,0000,PDQX083M,KJ1E03,Z07E,1000,6,0,865.0,1830.0,473.0,1
787,2FA00E8A,20201228,0000,PDQX083M,KJ1E03,Z07E,2000,7,2,976.0,1833.0,533.0,1
788,2FA01HYA,20201228,0000,PDQX083M,KJBE03,Z07E,1000,5,2,935.0,1944.0,481.0,1
789,2FA026TA,20201228,0000,PDQX083M,KJ1E03,Z07E,1000,3,7,1023.0,2409.0,425.0,1


# RWDC

In [37]:
#rwdc
inputParamGhl2Dati = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6600'],
                     ['60H0', 'G0H0']
                    ]
dfRwdc = get_ghl2_table('ccb_ci_rwdc', inputParamGhl2Dati, 
                        'hddsn, enddate, lhd, qualifier, phd, testcode, mfgid, hddtrial, band, rwdcmin, rwdcmax, \
                         row_number() over (partition by hddsn, qualifier, lhd, band order by enddate desc) as row_num')
dfRwdc = pd.read_pickle('df_ccb_ci_rwdc.pkl')
#dfRwdc.loc[:,['hddsn', 'lhd', 'phd', 'testcode', 'mfgid', 'band', 'rwdcmin', 'rwdcmax']]
dfRwdcFlat = pd.pivot_table(dfRwdc.query("row_num==1"),
                            index=['hddsn', 'mfgid', 'hddtrial', 'testcode', 'lhd'], 
                            columns=['qualifier', 'band'], 
                            values=['rwdcmin', 'rwdcmax'], 
                            aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfRwdcFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfRwdcFlat.columns)[5:]]
dfRwdcFlat.columns = cols
dfRwdcFlat.to_pickle('dfRwdcFlat.pkl')

current sql is select hddsn, enddate, lhd, qualifier, phd, testcode, mfgid, hddtrial, band, rwdcmin, rwdcmax,                          row_number() over (partition by hddsn, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_rwdc where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6600') and qualifier in ('60H0', 'G0H0') and pfcode not in ('9999', '99V6', '99V7') 
(8192, 12)


In [38]:
# Merge all tables

dfParamAll = dfHeadMap.copy(deep = True)
for df in [dfRsbnFlat, dfSerFlat, dfSerOffsetFlat, 
           dfOwFlat, dfMcwFlat, dfMrwFlat, dfAmpFlat, 
           dfAsymFlat, dfRwipFlat, dfSatiSerFlat, dfNmaxTempFlat, dfAsb]:
    dfParamAll = dfParamAll.merge(df,
                             left_on  = ['hddsn', 'lhd', 'phd', 'mfgid', 'testcode'],
                             right_on = ['hddsn', 'lhd', 'phd', 'mfgid', 'testcode'],
                             how='left',
                            )
    print(dfParamAll.shape)
    
for df in [dfSqmFlat, dfAdcFlat]:
    dfParamAll = dfParamAll.merge(df,
                             left_on  = ['hddsn', 'lhd', 'mfgid', 'testcode'],
                             right_on = ['hddsn', 'lhd', 'mfgid', 'testcode'],
                             how='left',
                            )
    print(dfParamAll.shape)

dfParamAll.to_pickle('dfParamAll.pkl')
dfParamAll.to_csv('dfParamAll.csv')

NameError: name 'dfSerFlat' is not defined

In [ ]:
pd.pivot_table(dfParamAll, index=['testcode', 'mfgid'], columns='lhd', values='hddsn', aggfunc='count')

# DET Data

In [15]:
# Association

inputParamVqaa = [snlist_csv, 
                  listTestcode,
                  listEnddt, 
                  listHddTrial,
                  ['6400']
                 ]
get_vqaa_table ('fact_hdd_headops', inputParamVqaa, 'product, slidersn, pheadno, hddsn, procid, mfgid, hddtrial, enddate, \
                 row_number() over ( partition by slidersn order by enddate desc) as row_num', whereAdded = None)

current sql is select product, slidersn, pheadno, hddsn, procid, mfgid, hddtrial, enddate,                  row_number() over ( partition by slidersn order by enddate desc) as row_num from vqaa.fact_hdd_headops where product='pdq' and testpgmver in ('PDQX083M') and enddt between '20201223' and '20210101' and hddtrial in ('Z07E', 'NPIF') and hddsn in ('2FA00WRA', '2FA02E8A', '2FA06EXA', '2FA06GNA', '2FA06H4A', '2FA06H1A', '2FA06G5A', '2FA06G3A', '2FA02DKA', '2FA01E4A', '2FA02EUA', '2FA02EAA', '2FA02DNA', '2FA01EHA') and procid in ('6400') 
(306, 9)


,product,slidersn,pheadno,hddsn,procid,mfgid,hddtrial,enddate,row_num
0,pdq,49FB51932F,7,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1
1,pdq,4AF083B607,4,2FA06G3A,6400,KJ1C03,NPIF,20201229143527,1
2,pdq,49FBF2D223,12,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1
3,pdq,49FBF1B432,0,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1
4,pdq,49F8D1182C,10,2FA06H4A,6400,KJ1C03,NPIF,20201229142426,1
...,...,...,...,...,...,...,...,...,...
301,pdq,49F491C709,7,2FA01EHA,6400,KJ1C03,Z07E,20201223225255,1
302,pdq,49F6A28B38,7,2FA02DNA,6400,KJ1C03,Z07E,20201223224858,1
303,pdq,4AFF60B71B,7,2FA06GNA,6400,KJ1C03,NPIF,20201229141542,1
304,pdq,4AFF60B71B,7,2FA06GNA,6400,KJ1C03,NPIF,20201228132753,2


In [19]:
## sdet ##
inputParamSldr = ['df_fact_hdd_headops.csv', 
                  ['2020-01-01', '2020-07-31'],
                 ]
get_sdet_table('sldr.sdet_special_trans', inputParamSldr, 
               'slidersn, product, storeday, \
                wew_r0_nm_sto_2, wew_r1_nm_sto_2, wew_r0_mww_nm_sto_2, wew_r1_mww_nm_sto_2, wew_r0_umrw_nm_sto_2, wew_r1_umrw_nm_sto_2, wew_r0_eb_nm_sto_2, wew_r1_eb_nm_sto_2, \
                row_number() over ( partition by slidersn order by storeday desc) as row_num',
                whereAdded = None)

current sql is select slidersn, product, storeday,                 wew_r0_nm_sto_2, wew_r1_nm_sto_2, wew_r0_mww_nm_sto_2, wew_r1_mww_nm_sto_2, wew_r0_umrw_nm_sto_2, wew_r1_umrw_nm_sto_2, wew_r0_eb_nm_sto_2, wew_r1_eb_nm_sto_2,                 row_number() over ( partition by slidersn order by storeday desc) as row_num from hive.sldr.sdet_special_trans where storeday between '2020-01-01' and '2020-07-31' and slidersn in ('49FB51932F', '4AF083B607', '49FBF2D223', '49FBF1B432', '49F8D1182C', '4AF081CC0A', '4AF081CC0A', '4AF081CC0A', '49F8D11835', '49FBF3FF22', '4AF0823428', '4AFF60B339', '49FBF3A615', '4AFF61130A', '4AFF61130A', '49F6A28627', '49F8D1183B', '49FBF0330B', '49F8818902', '4AF080AC1D', '4AF080AC1D', '4AF080AC1D', '4AFF60B338', '4AF0821A19', '4AFF605126', '49F491C70C', '49FBF2D221', '49FBF0F215', '49F491C715', '49FBF03510', '49FCC2130D', '49FCC2130D', '4AFF61591C', '4AFF61591C', '49F5508532', '49F5536420', '49F8A33C1D', '4A2B82810B', '49F4910703', '49FBF03B0F', '49FBF0351D', '4

,slidersn,product,storeday,wew_r0_nm_sto_2,wew_r1_nm_sto_2,wew_r0_mww_nm_sto_2,wew_r1_mww_nm_sto_2,wew_r0_umrw_nm_sto_2,wew_r1_umrw_nm_sto_2,wew_r0_eb_nm_sto_2,wew_r1_eb_nm_sto_2,row_num
0,4AF0823428,C3_B_D,2020-07-29,None,None,None,None,None,None,None,None,1
1,4AFF60B339,C3_T_D,2020-07-28,None,None,None,None,None,None,None,None,1
2,49F8A33C1D,C3_B_D,2020-03-10,None,None,None,None,None,None,None,None,1
3,4AFF61591C,C3_T_D,2020-07-30,None,None,None,None,None,None,None,None,1
4,49F8D11835,C3_B_D,2020-06-07,None,None,None,None,None,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...
86,49FBF2DD0F,C3_T_D,2020-03-10,None,None,None,None,None,None,None,None,1
0,49F491C709,C3_T_D,2020-03-11,None,None,None,None,None,None,None,None,1
1,49F6A28B38,C3_T_D,2020-03-12,None,None,None,None,None,None,None,None,1
2,4AFF60B71B,C3_T_D,2020-07-28,None,None,None,None,None,None,None,None,1


In [20]:
#Combine slider sn, pheadno, and HDD SN
df = pd.read_pickle('df_fact_hdd_headops.pkl').query("row_num==1").merge(pd.read_pickle('df_sldr.sdet_special_trans.pkl').loc[:,['slidersn', 'storeday', 'wew_r0_nm_sto_2', 'row_num']].query("row_num==1"),
                                                                         left_on=['slidersn', 'row_num'], 
                                                                         right_on=['slidersn', 'row_num'], 
                                                                         how='left')
df.to_pickle('hddops_association.pkl')
df

,product,slidersn,pheadno,hddsn,procid,mfgid,hddtrial,enddate,row_num,storeday,wew_r0_nm_sto_2
0,pdq,49FB51932F,7,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1,2020-03-09,None
1,pdq,4AF083B607,4,2FA06G3A,6400,KJ1C03,NPIF,20201229143527,1,2020-07-28,None
2,pdq,49FBF2D223,12,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1,2020-03-14,None
3,pdq,49FBF1B432,0,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1,2020-03-14,None
4,pdq,49F8D1182C,10,2FA06H4A,6400,KJ1C03,NPIF,20201229142426,1,2020-06-07,None
...,...,...,...,...,...,...,...,...,...,...,...
247,pdq,49FB530725,15,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1,2020-03-09,None
248,pdq,49F8A34E11,10,2FA02E8A,6400,KJ1C03,Z07E,20201223223939,1,2020-03-08,None
249,pdq,49F491C709,7,2FA01EHA,6400,KJ1C03,Z07E,20201223225255,1,2020-03-11,None
250,pdq,49F6A28B38,7,2FA02DNA,6400,KJ1C03,Z07E,20201223224858,1,2020-03-12,None
